In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from keras.optimizers import Adam

from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
params = {
    'epochs': 200,
    'test_size': 0.2,
    'random_state': 1337,
    'validation_split': 0.2
}

In [3]:
df = pd.read_csv("../data/qsc_out.random_scan_nfp2.csv")

In [4]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y0,y1,y2,y3,y4,y5,y6
0,-0.115912,-0.207162,0.001411,0.012060,0.000871,-0.000108,-0.736734,0.012462,0.783335,0.278748,0.497138,0.645087,0.926717,1.717088,0.338459
1,-0.081966,-0.182033,0.001298,0.010903,0.000813,-0.000155,-0.755056,0.031954,1.379462,0.284927,0.386816,0.493242,0.881144,1.562226,0.326036
2,-0.098121,0.188199,0.001285,-0.010709,0.000807,0.000152,-0.641071,0.060675,1.124535,0.342645,0.523383,0.639508,0.869696,1.574066,0.331869
3,-0.077109,-0.206706,0.001522,0.006428,0.000926,-0.000304,-0.868233,-0.092663,1.205836,0.265378,0.541464,0.512058,0.907885,1.711111,0.324205
4,-0.082828,0.221897,0.000230,-0.008468,0.000198,0.000174,-0.758676,-0.317667,1.026909,0.273752,0.751935,0.643160,0.944501,1.518423,0.326940


In [5]:
df.shape

(4796, 15)

In [6]:
x_columns = [col for col in df.columns if col.startswith('x')]
y_columns = [col for col in df.columns if col.startswith('y')]

Y = df[x_columns].values
X = df[y_columns].values

In [7]:
def preprocess_data(X_train, X_test, Y_train, Y_test, params):
    scaler_x = StandardScaler().fit(X_train)
    scaler_y = StandardScaler().fit(Y_train)
    X_train = scaler_x.transform(X_train)
    X_test = scaler_x.transform(X_test)
    Y_train = scaler_y.transform(Y_train)
    Y_test = scaler_y.transform(Y_test)

    input_shape = X_train.shape[1]
    
    output_shape = Y_train.shape[1]
    return X_train, X_test, Y_train, Y_test, input_shape, output_shape, scaler_x, scaler_y

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=params['test_size'], 
                                                    random_state=params['random_state'])

X_train, X_test, Y_train, Y_test, input_shape, output_shape, scaler_x, scaler_y = preprocess_data(X_train, X_test, Y_train, Y_test, params)

In [8]:
X_train.mean(axis=0), X_train.std(axis=0) 

(array([-4.01657042e-15,  4.72944589e-15,  2.50063555e-15, -3.41465936e-15,
        -5.97028509e-15, -9.70058815e-15, -3.51141604e-14]),
 array([1., 1., 1., 1., 1., 1., 1.]))

## Appears to be drift, perhaps the sample is not big enough

In [9]:
X_test.mean(axis=0), X_test.std(axis=0) 

(array([0.03647091, 0.05036899, 0.0240539 , 0.01615513, 0.02262298,
        0.069791  , 0.00742891]),
 array([1.04363768, 1.03365372, 1.03354061, 1.01573765, 1.04018519,
        0.98510014, 1.0395029 ]))

In [10]:
Y_train.mean(axis=0), Y_train.std(axis=0) 

(array([ 3.83484230e-17, -2.05489663e-17,  1.57677139e-16,  1.15291278e-16,
        -6.44832351e-17, -3.98823600e-17,  2.63764795e-16,  9.18495381e-16]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]))

In [11]:
Y_test.mean(axis=0), Y_test.std(axis=0)

(array([-0.02525043,  0.00971201,  0.01397497,  0.04719729,  0.01297527,
        -0.02875165, -0.00224075,  0.00415783]),
 array([1.00535971, 1.00234493, 1.04188367, 1.01245292, 1.03768115,
        0.98217117, 1.014723  , 1.01594242]))

## Dummy regressor

In [12]:
from sklearn.dummy import DummyRegressor

In [13]:
regr = MultiOutputRegressor(DummyRegressor(strategy="mean")).fit(X_train, Y_train)
regr.predict(X_train)

array([[ 3.14891202e-17, -9.26150594e-18,  1.42627191e-16, ...,
        -4.35290779e-17,  2.74140576e-16,  9.03922980e-16],
       [ 3.14891202e-17, -9.26150594e-18,  1.42627191e-16, ...,
        -4.35290779e-17,  2.74140576e-16,  9.03922980e-16],
       [ 3.14891202e-17, -9.26150594e-18,  1.42627191e-16, ...,
        -4.35290779e-17,  2.74140576e-16,  9.03922980e-16],
       ...,
       [ 3.14891202e-17, -9.26150594e-18,  1.42627191e-16, ...,
        -4.35290779e-17,  2.74140576e-16,  9.03922980e-16],
       [ 3.14891202e-17, -9.26150594e-18,  1.42627191e-16, ...,
        -4.35290779e-17,  2.74140576e-16,  9.03922980e-16],
       [ 3.14891202e-17, -9.26150594e-18,  1.42627191e-16, ...,
        -4.35290779e-17,  2.74140576e-16,  9.03922980e-16]])

In [14]:
mean_squared_error(Y_train, regr.predict(X_train))

1.0000000000000007

In [15]:
mean_squared_error(Y_test, regr.predict(X_test))

1.0291802816467932

## Train a linear regression for debugging

In [16]:
regr = MultiOutputRegressor(Ridge(random_state=123)).fit(X_train, Y_train)
regr.predict(X_train)

array([[ 0.15676934,  0.04012469, -0.12659159, ...,  0.06290221,
        -0.40890284,  0.03899315],
       [ 0.098538  ,  0.02856683, -0.02685268, ..., -0.01454209,
        -0.16032767,  0.49703159],
       [-0.17443072,  0.00250346,  0.13851425, ..., -0.0678374 ,
        -0.04978863,  0.27261235],
       ...,
       [ 0.02097198,  0.03903598,  0.03713012, ..., -0.02695103,
        -0.3743756 ,  0.34646245],
       [-0.26866067,  0.03050944,  0.16463002, ..., -0.05755192,
        -0.21087286,  0.37635694],
       [ 0.33351338,  0.08189727, -0.20413084, ...,  0.07795407,
        -0.54412842,  0.25056948]])

In [17]:
mean_squared_error(Y_train, regr.predict(X_train))

0.9248479543380279

In [18]:
mean_squared_error(Y_test, regr.predict(X_test))

0.955048583856655

## Simplest neural network

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [20]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(input_shape, activation="relu", name="layer_in"),
        layers.Dense(128, activation="relu", name="layer2"),
        layers.Dense(64, activation="relu", name="layer3"),
        layers.Dense(output_shape, name="layer_out"),
    ]
)

model(X_train[0:1])

<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[ 0.06464694, -0.00889271,  0.06453516,  0.0415666 ,  0.01000295,
         0.05986639, -0.02833799,  0.05403368]], dtype=float32)>

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_in (Dense)            (1, 7)                    56        
                                                                 
 layer2 (Dense)              (1, 128)                  1024      
                                                                 
 layer3 (Dense)              (1, 64)                   8256      
                                                                 
 layer_out (Dense)           (1, 8)                    520       
                                                                 
Total params: 9856 (38.50 KB)
Trainable params: 9856 (38.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [23]:
print("Fit model on training data")
history = model.fit(
    X_train,
    Y_train,
    batch_size=64,
    epochs=80,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_test, Y_test),
)

Fit model on training data
Epoch 1/80
60/60 [==============================] - 0s 2ms/step - loss: 0.9623 - mean_squared_error: 0.9623 - val_loss: 0.9661 - val_mean_squared_error: 0.9661
Epoch 2/80
60/60 [==============================] - 0s 799us/step - loss: 0.9294 - mean_squared_error: 0.9294 - val_loss: 0.9528 - val_mean_squared_error: 0.9528
Epoch 3/80
60/60 [==============================] - 0s 681us/step - loss: 0.9170 - mean_squared_error: 0.9170 - val_loss: 0.9435 - val_mean_squared_error: 0.9435
Epoch 4/80
60/60 [==============================] - 0s 690us/step - loss: 0.9082 - mean_squared_error: 0.9082 - val_loss: 0.9370 - val_mean_squared_error: 0.9370
Epoch 5/80
60/60 [==============================] - 0s 686us/step - loss: 0.8998 - mean_squared_error: 0.8998 - val_loss: 0.9410 - val_mean_squared_error: 0.9410
Epoch 6/80
60/60 [==============================] - 0s 715us/step - loss: 0.8939 - mean_squared_error: 0.8939 - val_loss: 0.9296 - val_mean_squared_error: 0.9296
Epo

60/60 [==============================] - 0s 637us/step - loss: 0.8087 - mean_squared_error: 0.8087 - val_loss: 0.8883 - val_mean_squared_error: 0.8883
Epoch 52/80
60/60 [==============================] - 0s 663us/step - loss: 0.8080 - mean_squared_error: 0.8080 - val_loss: 0.8876 - val_mean_squared_error: 0.8876
Epoch 53/80
60/60 [==============================] - 0s 662us/step - loss: 0.8061 - mean_squared_error: 0.8061 - val_loss: 0.8858 - val_mean_squared_error: 0.8858
Epoch 54/80
60/60 [==============================] - 0s 686us/step - loss: 0.8058 - mean_squared_error: 0.8058 - val_loss: 0.8821 - val_mean_squared_error: 0.8821
Epoch 55/80
60/60 [==============================] - 0s 693us/step - loss: 0.8033 - mean_squared_error: 0.8033 - val_loss: 0.8910 - val_mean_squared_error: 0.8910
Epoch 56/80
60/60 [==============================] - 0s 638us/step - loss: 0.8034 - mean_squared_error: 0.8034 - val_loss: 0.8803 - val_mean_squared_error: 0.8803
Epoch 57/80
60/60 [===============

In [24]:
mean_squared_error(Y_train, model.predict(X_train))

120/120 [==============================] - 0s 318us/step


0.76772865586145

In [25]:
mean_squared_error(Y_test, model.predict(X_test))

30/30 [==============================] - 0s 378us/step


0.8776152117416222